In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from datasetB_main import MyDataset

In [2]:
builder = MyDataset()
builder.download_and_prepare()
dataset_train_raw = builder.as_dataset(split='train')
dataset_test_raw = builder.as_dataset(split='test')
dataset_info = builder.info

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\damie\tensorflow_datasets\my_dataset\incomplete.CHYLTS_1.0.0\my_dataset-train.tfrecord*...:…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling C:\Users\damie\tensorflow_datasets\my_dataset\incomplete.CHYLTS_1.0.0\my_dataset-test.tfrecord*...: …

Dataset my_dataset downloaded and prepared to C:\Users\damie\tensorflow_datasets\my_dataset\1.0.0. Subsequent calls will reuse this data.


In [3]:
print('Raw train dataset:', dataset_train_raw)
print('Raw train dataset size:', len(dataset_train_raw), '\n')

print('Raw test dataset:', dataset_test_raw)
print('Raw test dataset size:', len(dataset_test_raw), '\n')

Raw train dataset: <_PrefetchDataset element_spec={'image': TensorSpec(shape=(640, 480, 3), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>
Raw train dataset size: 3359 

Raw test dataset: <_PrefetchDataset element_spec={'image': TensorSpec(shape=(640, 480, 3), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>
Raw test dataset size: 720 



In [4]:
NUM_TRAIN_EXAMPLES = dataset_info.splits['train'].num_examples
NUM_TEST_EXAMPLES = dataset_info.splits['test'].num_examples
NUM_CLASSES = dataset_info.features['label'].num_classes

print('Number of TRAIN examples:', NUM_TRAIN_EXAMPLES)
print('Number of TEST examples:', NUM_TEST_EXAMPLES)
print('Number of label classes:', NUM_CLASSES)

Number of TRAIN examples: 3359
Number of TEST examples: 720
Number of label classes: 3


In [5]:
INPUT_IMG_SIZE_ORIGINAL = dataset_info.features['image'].shape[0]
INPUT_IMG_SHAPE_ORIGINAL = dataset_info.features['image'].shape

INPUT_IMG_SIZE_REDUCED = INPUT_IMG_SIZE_ORIGINAL // 2
INPUT_IMG_SHAPE_REDUCED = (
    INPUT_IMG_SIZE_REDUCED,
    INPUT_IMG_SIZE_REDUCED,
    INPUT_IMG_SHAPE_ORIGINAL[2]
)

# Here we may switch between bigger or smaller image sized that we will train our model on.
INPUT_IMG_SIZE = INPUT_IMG_SIZE_REDUCED
INPUT_IMG_SHAPE = INPUT_IMG_SHAPE_REDUCED

print('Input image size (original):', INPUT_IMG_SIZE_ORIGINAL)
print('Input image shape (original):', INPUT_IMG_SHAPE_ORIGINAL)
print('\n')
print('Input image size (reduced):', INPUT_IMG_SIZE_REDUCED)
print('Input image shape (reduced):', INPUT_IMG_SHAPE_REDUCED)
print('\n')
print('Input image size:', INPUT_IMG_SIZE)
print('Input image shape:', INPUT_IMG_SHAPE)

Input image size (original): 640
Input image shape (original): (640, 480, 3)


Input image size (reduced): 320
Input image shape (reduced): (320, 320, 3)


Input image size: 320
Input image shape: (320, 320, 3)


In [8]:
def format_example(features):
    image = features['image']
    label = features['label']
    # Make image color values to be float.
    image = tf.cast(image, tf.float32)
    # Make image color values to be in [0..1] range.
    image = image / 255.
    # Make sure that image has a right size
    image = tf.image.resize(image, [INPUT_IMG_SIZE, INPUT_IMG_SIZE])
    return image, label

In [9]:
print(dataset_train_raw)
dataset_train = dataset_train_raw.map(format_example)
dataset_test = dataset_test_raw.map(format_example)

<_PrefetchDataset element_spec={'image': TensorSpec(shape=(640, 480, 3), dtype=tf.uint8, name=None), 'label': TensorSpec(shape=(), dtype=tf.int64, name=None)}>


In [10]:
# Explore what values are used to represent the image.
(first_image, first_lable) = list(dataset_train.take(1))[0]
print('Label:', first_lable.numpy(), '\n')
print('Image shape:', first_image.numpy().shape, '\n')
print(first_image.numpy())

Label: 2 

Image shape: (320, 320, 3) 

[[[0.9843137  0.9764706  0.98039216]
  [0.98039216 0.98039216 0.98039216]
  [0.98039216 0.98039216 0.98039216]
  ...
  [0.3509804  0.327451   0.2647059 ]
  [0.33431375 0.32254905 0.25588238]
  [0.33235294 0.32058823 0.25392157]]

 [[0.9843137  0.9764706  0.98039216]
  [0.98333335 0.97745097 0.98039216]
  [0.98039216 0.98039216 0.98039216]
  ...
  [0.3421569  0.327451   0.2617647 ]
  [0.32843137 0.31666666 0.25      ]
  [0.32352942 0.31176472 0.23921569]]

 [[0.9843137  0.9764706  0.98039216]
  [0.9843137  0.9764706  0.98039216]
  [0.98039216 0.98039216 0.98039216]
  ...
  [0.33186275 0.32009804 0.25343138]
  [0.3254902  0.3137255  0.24509804]
  [0.32107845 0.30931374 0.23480392]]

 ...

 [[0.7980392  0.7745098  0.7843138 ]
  [0.79068625 0.775      0.7769608 ]
  [0.81666666 0.8009804  0.7970588 ]
  ...
  [0.0627451  0.05490196 0.05882353]
  [0.0627451  0.05490196 0.05882353]
  [0.0627451  0.05490196 0.05882353]]

 [[0.8539216  0.8313726  0.8480392

In [13]:
#Load model
model = tf.keras.models.load_model('rock_paper_scissors_cnn.keras')
model.summary()
print(model.input_shape)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 148, 148, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 74, 74, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 72, 72, 64)     │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 36, 36, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 34, 34, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 17, 17, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 15, 15, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 7, 7, 128)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 6272)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │     3,211,776 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 3)              │         1,539 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 6,946,952 (26.50 MB)

 Trainable params: 3,473,475 (13.25 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 3,473,477 (13.25 MB)

(None, 150, 150, 3)


In [12]:
# Test model with our dataset
model.evaluate(dataset_test)

ValueError: Exception encountered when calling Sequential.call().

[1mInvalid input shape for input Tensor("IteratorGetNext:0", shape=(320, 320, 3), dtype=float32). Expected shape (None, 150, 150, 3), but input has incompatible shape (320, 320, 3)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(320, 320, 3), dtype=float32)
  • training=False
  • mask=None